# Introducing defects in structures from MP with JAVIS

In this notebook we will study how we can use two different APIs together and see if we can find a way to automate inducing defects in a structure. 

A few problems that _has_ to be solved
- Make JAVIS automatically read input in POSCAR-format from MP. 
- Jarvis identifies possible defects with its Wyatt position/number/character. Does Jarvis introduce correct defects, and if so how many? If some of the induced defects are correct, what happens to the ones that are correctly identified and those who are not? 
- After introducing defects in a structure, can we then predict properties with ML of different APIs? Predict stability with Open Quantum Materials Database (oqmd)? Automatically run in Vasp? 
    
To start off, we can have a look at how far it is possible to come without answering the questions while hard-coding. 

In [26]:
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
mpdr = MPDataRetrieval("b7RtVfJTsUg6TK8E")
df = mpdr.get_dataframe(criteria={"pretty_formula": "SiC", "e_above_hull": {"$lt": 1e-6}}, 
                        properties=["density", "pretty_formula", "icsd_id", "band_gap", "structure"])
(df.structure[0])

Structure Summary
Lattice
    abc : 3.094654883412527 3.094654883412527 25.3091
 angles : 90.0 90.0 120.00001888614457
 volume : 209.9093083831296
      A : 1.547327 -2.68005 0.0
      B : 1.547327 2.68005 0.0
      C : 0.0 0.0 25.3091
PeriodicSite: Si (0.0000, 0.0000, 5.0597) [0.0000, 0.0000, 0.1999]
PeriodicSite: Si (0.0000, 0.0000, 10.1225) [0.0000, 0.0000, 0.4000]
PeriodicSite: Si (1.5473, 0.8934, 7.5922) [0.3333, 0.6667, 0.3000]
PeriodicSite: Si (0.0000, 0.0000, 15.1844) [0.0000, 0.0000, 0.6000]
PeriodicSite: Si (1.5473, 0.8934, 25.3083) [0.3333, 0.6667, 1.0000]
PeriodicSite: Si (1.5473, 0.8934, 17.7159) [0.3333, 0.6667, 0.7000]
PeriodicSite: Si (1.5473, -0.8934, 22.7758) [0.6667, 0.3333, 0.8999]
PeriodicSite: Si (0.0000, 0.0000, 20.2479) [0.0000, 0.0000, 0.8000]
PeriodicSite: Si (1.5473, -0.8934, 2.5313) [0.6667, 0.3333, 0.1000]
PeriodicSite: Si (1.5473, -0.8934, 12.6538) [0.6667, 0.3333, 0.5000]
PeriodicSite: C (1.5473, 0.8934, 19.6191) [0.3333, 0.6667, 0.7752]
PeriodicSite: C (

In [12]:
### Hardcoding the above

from jarvis.core.atoms import Atoms
#Silicon crystal structure example. These crystal-structure data are generally obtained from XRD-type experiments.
box = [[3.09465488, 0, 0], [-1.547327, 2.680050, 0.000000], [0, 0, 25.30910000]]
coords = [[0.00000000, 0.00000000, 0.19991800], [0.00000000, 0.00000000, 0.39995400], [0.33333300, 0.66666700, 0.29998100], [0.00000000, 0.00000000, 0.59995900], 
          [0.33333300, 0.66666700, 0.99996900], [0.33333300, 0.66666700, 0.69998000], [0.66666700, 0.33333300, 0.89990500], [0.00000000,  0.00000000,  0.80002300], 
          [0.66666700, 0.33333300, 0.10001600], [0.66666700, 0.33333300, 0.49997000], [0.33333300, 0.66666700, 0.77518100], [0.33333300,  0.66666700,  0.07517800],
          [0.00000000,  0.00000000,  0.67489000], [0.00000000,  0.00000000,  0.27492800], [0.66666700,  0.33333300,  0.17498900], [0.66666700,  0.33333300,  0.57517800], 
          [0.00000000,  0.00000000,  0.47488900], [0.66666700,  0.33333300,  0.97491200], [0.33333300,  0.66666700,  0.37517600], [0.00000000,  0.00000000,  0.87500400]]
elements = ["Si", "Si", "Si", "Si", "Si", "Si", "Si", "Si","Si","Si", "C","C","C", "C", "C", "C", "C", "C", "C", "C"]
SiC = Atoms(lattice_mat=box, coords=coords, elements=elements)

In [13]:
print ('volume',SiC.volume)
print ('density in g/cm3', SiC.density)
print ('composition as dictionary', SiC.composition)
print ('Chemical formula', SiC.composition.reduced_formula)
print ('Spacegroup info', SiC.spacegroup())
print ('lattice-parameters', SiC.lattice.abc, SiC.lattice.angles)
print ('packing fraction',SiC.packing_fraction)
print ('number of atoms',SiC.num_atoms)
print ('Center of mass', SiC.get_center_of_mass())
print ('Atomic number list', SiC.atomic_numbers)

volume 209.90936807322464
density in g/cm3 3.171909122453925
composition as dictionary OrderedDict([('Si', 10), ('C', 10)])
Chemical formula SiC
Spacegroup info P3m1 (156)
lattice-parameters [3.09465, 3.094656, 25.3091] [90.0, 90.0, 120.0]
packing fraction 0.33405
number of atoms 20
Center of mass [ 0.464196    0.804015   13.73014356]
Atomic number list [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [16]:
print (SiC)

Si0C0
1.0
3.09465488 0.0 0.0
-1.547327 2.68005 0.0
0.0 0.0 25.3091
Si C
10 10
Cartesian
0.0 0.0 5.0597446538
1.54732488066 0.89334910665 2.5313149456
0.0 0.0 20.2478621093
1.54732488066 0.89334910665 22.7757856355
-4.88065999997434e-06 1.7867008933500002 17.715863818000003
1.54732488066 0.89334910665 12.653790727
0.0 0.0 15.184422326900002
-4.88065999997434e-06 1.7867008933500002 7.5922491271000005
0.0 0.0 10.1224757814
-4.88065999997434e-06 1.7867008933500002 25.3083154179
-4.88065999997434e-06 1.7867008933500002 9.4953669016
-4.88065999997434e-06 1.7867008933500002 19.6191334471
-4.88065999997434e-06 1.7867008933500002 1.9026875198
0.0 0.0 17.080858499
0.0 0.0 6.9581802448
1.54732488066 0.89334910665 4.4288140999
1.54732488066 0.89334910665 14.5572375198
0.0 0.0 12.0190131899
1.54732488066 0.89334910665 24.6741452992
0.0 0.0 22.1455637364



In [17]:
from jarvis.analysis.defects.vacancy import Vacancy

vacs = Vacancy(atoms=SiC).generate_defects(enforce_c_size=10.0) #enforces cell-size to be close to 10 Angstroms
#len(vacs), vacs[0].to_dict()["defect_structure"].num_atoms

In [6]:
len(vacs) 
# We find that there are three unique points vacanc available based on Wycoff-position information

3

In [23]:
for i in range(len(vacs)): 
  print(vacs[i].to_dict()['symbol'], vacs[i]._wyckoff_multiplicity, vacs[i]._symbol,
        len(vacs[i].to_dict()['atoms']['coords']), 
        len(vacs[i].to_dict()['defect_structure']['coords']))

Si a Si 320 319
Si b Si 320 319
Si c Si 320 319


In [8]:
print(vacs[0])

OrderedDict([ ( 'atoms',
                OrderedDict([ ( 'lattice_mat',
                                [ [12.37861952, 0.0, 0.0],
                                  [-6.189308, 10.7202, 0.0],
                                  [0.0, 0.0, 25.3091]]),
                              ( 'coords',
                                [ [0.0, 0.0, 0.199918],
                                  [0.0, 0.25, 0.199918],
                                  [0.0, 0.5, 0.199918],
                                  [0.0, 0.75, 0.199918],
                                  [0.25, 0.0, 0.199918],
                                  [0.25, 0.25, 0.199918],
                                  [0.25, 0.5, 0.199918],
                                  [0.25, 0.75, 0.199918],
                                  [0.5, 0.0, 0.199918],
                                  [0.5, 0.25, 0.199918],
                                  [0.5, 0.5, 0.199918],
                                  [0.5, 0.75, 0.199918],
                           